In [ ]:
#os.chdir('/content/drive/MyDrive/Escuela/Doctorado/Projects/ConvNext/ConvNext_V2')

In [4]:
!nvidia-smi

Tue Oct 21 12:32:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.65.06              Driver Version: 580.65.06      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    Off |   00000000:01:00.0 Off |                  N/A |
|  0%   41C    P8             10W /  220W |      18MiB /  12282MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_allocated()  
torch.cuda.memory_reserved()  
import zipfile
import os
import nibabel as nib
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from torch.utils.data import random_split
import wandb
from torchvision import models
import torch.optim as optim

# DataLoader

In [6]:
%run src/ConvNext_models.ipynb
%run src/metrics.ipynb
%run src/hg_lossfunctions.ipynb
%run src/trainer.ipynb
%run src/BratsDataloader.ipynb
%run src/Optimizer.ipynb

Output: torch.Size([2, 1, 512, 512])
Path BrATS2021: BrATS2021/
Path BrATS2023: BrATS2023/


In [7]:
base_path ="../BRATS2024/"

transform = transforms.Compose([
    transforms.Resize((512, 512))  # Cambia al tamaño deseado
    #transforms.Normalize(mean=[0.5], std=[0.5]) # For -1 a 1
])

# Crear el dataset y el DataLoader
dataset = BRATSDataset_2(base_path, img_transform=transform, mask_transform = transform)

train_size = int(0.7 * len(dataset))  
val_size   = int(0.15 * len(dataset))  
test_size  = len(dataset) - train_size - val_size  

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

BATCH_SIZE = 6
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size   = BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size  = BATCH_SIZE, shuffle=False)
print(np.shape(train_loader))
for img, mask in train_loader:
    print(np.shape(img))
    print(np.shape(mask))
    break

def get_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

def get_model_memory_size(model):
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())
    return param_size
def get_model_flops(model, input_size=(1, 512, 512), device='cuda'):
    model = model.to(device)
    model.eval()
    
    total_flops = 0
    total_params = sum(p.numel() for p in model.parameters())

()
torch.Size([6, 1, 512, 512])
torch.Size([6, 1, 512, 512])


In [8]:
from matplotlib.backends.backend_pdf import PdfPages

device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes     = 1  # Ejemplo: edema, realce, necrosis, etc.
ConvNext_models = {  #'HyperNetSegmentation'    : HyperNetSegmentation,
                     'ConvNeXtSegmentationMF'  : ConvNeXtSegmentationMF, # dió IoU: 0.8
                     #'ConvNeXtUNet'            :prueba_borrar ConvNeXtUNet,
                     #'ConvNeXtUNet_B'          : ConvNeXtUNet_B,
                     #ConvNext_Base'           : ConvNeXtSegmentation
                     #'ConvNeXtSegmentationRA'  : ConvNeXtSegmentationRA #use highg resources
}

optimizers      =  { 
                     #'Adam'     : lambda model: optim.Adam(model.parameters(), lr = 0.001),
                     'AdamW'    : lambda model: optim.AdamW(model.parameters(), lr=0.001),
                     #'CRTAdamW' : lambda model:CRTAdamW(model.parameters(), lr = 0.001),
                  }

functions_losses = { #'BCEDiceLoss': BCEDiceLoss(),
                     #'DSC': DiceLoss(),
                     'BCE':nn.BCEWithLogitsLoss()
                   }

for model_name, model_class in ConvNext_models.items():      # Models 
    for optimizer_name, optimizer_class in optimizers.items(): # Optimizers
        for losses_name, losses_class in functions_losses.items(): # Losses
            print('------------------------ Model Name:', model_name)
            print('------------------------ OPtimizer Name:', optimizer_name)
            print('------------------------ Loss Name:', losses_name)

            if model_name =='ConvNeXtUNet' or  model_name == 'ConvNeXtRAUNet' or model_name == 'ConvNeXtMFRAUNet':
                model = model_class(num_classes, backbone_type='base')
            else:
                model       =  model_class(num_classes)   #ConvNeXtSegmentation(num_classes)
            
            model       = model.to(device)  # Si estás usando una GPU
            criterion   =  losses_class #nn.BCEWithLogitsLoss()  # CrossEntropyLoss()
            optimizer   =  optimizer_class(model)# CRTAdamW(model.parameters(), lr = 0.001)#optim.AdamW(model.parameters(), lr=0.001)# CRTAdamW
            num_epochs  = 100
        
            Metrics = []
            Conv_Next_trainer = UNetTrainer(
                    model        = model,
                    device       = device,
                    optimizer    = optimizer,
                    criterion    = criterion,
                    train_loader = train_loader,
                    val_loader   = val_loader,
                    test_loader  = test_loader,
                    epochs       = num_epochs,
                    filename_model = model_name + '.pth'
                )
            total_params      = get_model_parameters(model)
            model_memory_size = get_model_memory_size(model)/ (1024 ** 2)
            FLOps             = get_model_flops(model)
            print(FLOps)
            wandb.init(project="ConvNext_V2_RTX4070Super_Brats2024_", config={
                'Dataset'       : base_path, 
                'device'        : device,
                'model name'    : model_name,
                'epochs'        : num_epochs,
                'batch size'    : BATCH_SIZE,
                'loss_function' : losses_name, #criterion,
                'optimizer'     : optimizer_name,#optimizer,
                'total params'  :total_params,
                'Model Size (MB)'   : model_memory_size
            })
            # Start training
            Conv_Next_trainer.train()
            # Test the model if needed
            L_Net_images, L_Net_preds, L_Net_targets = Conv_Next_trainer.test()
            # Visualize results
            final_file = "BRATS2024_"+model_name+"_"+ optimizer_name+"_"+losses_name+'.pdf'
            Conv_Next_trainer.visualize_results(L_Net_images, L_Net_targets, L_Net_preds, num_samples=10, save_path= final_file)
            for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
                metrics = Conv_Next_trainer.evaluate_metrics(threshold=th)
                Metrics.append(metrics)
        
            wandb.finish()

------------------------ Model Name: ConvNeXtSegmentationMF
------------------------ OPtimizer Name: AdamW
------------------------ Loss Name: BCE
None


wandb: Currently logged in as: fd-hernandezgutierrez (hgfernandod) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch [1/100] - Train Loss: 0.2878, Val Loss: 0.1716
New best model saved with Val Loss: 0.1716
Epoch [2/100] - Train Loss: 0.1427, Val Loss: 0.1482
New best model saved with Val Loss: 0.1482
Epoch [3/100] - Train Loss: 0.1259, Val Loss: 0.0975
New best model saved with Val Loss: 0.0975
Epoch [4/100] - Train Loss: 0.1096, Val Loss: 0.1156
Epoch [5/100] - Train Loss: 0.1118, Val Loss: 0.0864
New best model saved with Val Loss: 0.0864
Epoch [6/100] - Train Loss: 0.0918, Val Loss: 0.0774
New best model saved with Val Loss: 0.0774
Epoch [7/100] - Train Loss: 0.0738, Val Loss: 0.0531
New best model saved with Val Loss: 0.0531
Epoch [8/100] - Train Loss: 0.0730, Val Loss: 0.0612
Epoch [9/100] - Train Loss: 0.0758, Val Loss: 0.0947
Epoch [10/100] - Train Loss: 0.0563, Val Loss: 0.0478
New best model saved with Val Loss: 0.0478
Epoch [11/100] - Train Loss: 0.0471, Val Loss: 0.0707
Epoch [12/100] - Train Loss: 0.0506, Val Loss: 0.0519
Epoch [13/100] - Train Loss: 0.0493, Val Loss: 0.0447
New be

PermissionError: [Errno 1] Operation not permitted: '/media/fernando/KINGSTON/to_GitHub/BRATS2024_ConvNeXtSegmentationMF_AdamW_BCE.pdf' -> '/media/fernando/KINGSTON/to_GitHub/wandb/run-20251021_123219-ncqx3b5d/files/BRATS2024_ConvNeXtSegmentationMF_AdamW_BCE.pdf'

# Fase de pruebas usando K-Fold y graficar distribución de violin

In [ ]:

device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes     = 1  # Ejemplo: edema, realce, necrosis, etc.
ConvNext_models = {  #'HyperNetSegmentation'    : HyperNetSegmentation,
                     'ConvNeXtSegmentationMF'  : ConvNeXtSegmentationMF#, # dió IoU: 0.8
                     #'ConvNeXtUNet'            : ConvNeXtUNet,
                     #'ConvNeXtUNet_B'          : ConvNeXtUNet_B,
                     #'ConvNext_Base'           : ConvNeXtSegmentation
                     #'ConvNeXtSegmentationRA'  : ConvNeXtSegmentationRA #use highg resources
}

optimizers      =  { #'CRTAdamW' : lambda model:CRTAdamW(model.parameters(), lr = 0.001),
                     'Adam'     : lambda model: optim.Adam(model.parameters(), lr = 0.001),
                     'AdamW'    : lambda model: optim.AdamW(model.parameters(), lr=0.001)
                    }

functions_losses = { 'BCEDiceLoss': BCEDiceLoss(),
                     'DSC': DiceLoss(),
                     'BCE':nn.BCEWithLogitsLoss()
                   }

In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import wandb

def train_models_violin(
    ConvNext_models,
    optimizers,
    functions_losses,
    train_loader,
    val_loader,
    test_loader,
    num_classes=1,
    device=None,
    num_epochs=100,
    BATCH_SIZE=4,
    use_kfold=False,
    k=5,
    metrics_to_plot=None   # <- lista opcional de métricas a graficar
):
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = train_loader.dataset

    all_metrics_data = []

    for model_name, model_class in ConvNext_models.items():
        for optimizer_name, optimizer_class in optimizers.items():
            for losses_name, losses_class in functions_losses.items():
                
                print('------------------------ Model:', model_name)
                print('------------------------ Optimizer:', optimizer_name)
                print('------------------------ Loss:', losses_name)

                metrics_per_model = []

                if use_kfold:
                    kf = KFold(n_splits=k, shuffle=True, random_state=42)
                    fold_iter = enumerate(kf.split(dataset))
                else:
                    fold_iter = [(0, (range(len(dataset)), range(len(val_loader.dataset))))]

                for fold, (train_idx, val_idx) in fold_iter:
                    print(f"===== Fold {fold+1} =====")

                    if use_kfold:
                        train_subset = Subset(dataset, train_idx)
                        val_subset   = Subset(dataset, val_idx)
                        train_loader_fold = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
                        val_loader_fold   = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
                    else:
                        train_loader_fold = train_loader
                        val_loader_fold   = val_loader

                    # Instanciar modelo
                    if model_name in ['ConvNeXtUNet', 'ConvNeXtRAUNet', 'ConvNeXtMFRAUNet']:
                        model = model_class(num_classes, backbone_type='base')
                    else:
                        model = model_class(num_classes)
                    model = model.to(device)

                    criterion = losses_class
                    optimizer = optimizer_class(model)

                    trainer = UNetTrainer(
                        model        = model,
                        device       = device,
                        optimizer    = optimizer,
                        criterion    = criterion,
                        train_loader = train_loader_fold,
                        val_loader   = val_loader_fold,
                        test_loader  = test_loader,
                        epochs       = num_epochs,
                        filename_model = f"{model_name}_fold{fold+1}.pth"
                    )

                    wandb.init(project="ConvNext_V2_Kfold_RTX4070Super", config={
                        'Dataset'       : 'CustomDataset', 
                        'device'        : device,
                        'model name'    : model_name,
                        'epochs'        : num_epochs,
                        'batch size'    : BATCH_SIZE,
                        'loss_function' : losses_name,
                        'optimizer'     : optimizer_name
                    })

                    trainer.train()
                    L_Net_images, L_Net_preds, L_Net_targets = trainer.test()
                    # Visualize results
                    final_file = model_name+"_"+ optimizer_name+"_"+losses_name+"_k_"+fold+'.pdf'
                    trainer.visualize_results(L_Net_images, L_Net_targets, L_Net_preds, num_samples=5, save_path= final_file)
                    metrics_fold = trainer.evaluate_metrics(threshold=0.2)

                    # Guardar métricas
                    for metric_name, value in metrics_fold.items():
                        metrics_per_model.append({
                            "Model": model_name,
                            "Optimizer": optimizer_name,
                            "Loss": losses_name,
                            "Fold": fold+1,
                            "Metric": metric_name,
                            "Value": value
                        })
                        all_metrics_data.append({
                            "Model": model_name,
                            "Optimizer": optimizer_name,
                            "Loss": losses_name,
                            "Fold": fold+1,
                            "Metric": metric_name,
                            "Value": value
                        })

                    wandb.finish()

                # Crear DataFrame solo para este modelo y configuración
                df_model = pd.DataFrame(metrics_per_model)

                # Filtrar solo métricas deseadas si se pasa la lista
                if metrics_to_plot is not None:
                    df_model = df_model[df_model['Metric'].isin(metrics_to_plot)]

                # Violin plot
                plt.figure(figsize=(12,6))
                sns.boxplot(
                    x="Metric",
                    y="Value",
                    hue="Loss",   # opcional: hue="Optimizer"
                    data=df_model
                )
                #sns.violinplot(
                #    x     = "Metric",
                #    y     = "Value",
                #    hue   = "Loss",   # opcional: hue="Optimizer"
                #    data  = df_model,
                #    split = True,
                #    inner = None, #quartile"
                #    bw    = 0.5,           # ajusta el ancho del KDE (kernel density)
                #    cut   = 0
                #)
                plt.title(f"Distribución de métricas para {model_name} ({optimizer_name}, {losses_name})")
                plt.ylabel("Valor de la Métrica")
                plt.xlabel("Métricas")
                plt.xticks(rotation=45)
                plt.legend(title="Loss")
                plt.show()

    return pd.DataFrame(all_metrics_data)



In [ ]:
df_metrics = train_models_violin(
    ConvNext_models=ConvNext_models,
    optimizers=optimizers,
    functions_losses=functions_losses,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    num_classes=1,
    device=device,
    num_epochs=100,
    BATCH_SIZE=6,
    use_kfold=True,
    k=5,
    metrics_to_plot=["Dice Coefficient", "IoU Score", "Sensitivity", "Specificity",'Accuracy','F1-Score']
)